In [38]:
import asyncio
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import nest_asyncio
import os
import json
from datetime import datetime
import csv
from datetime import datetime

In [36]:
# Inisialisasi driver Chrome
driver = webdriver.Chrome()
# Buka halaman web
driver.get('https://x.com')
# Maksimalkan ukuran jendela browser
driver.maximize_window()

In [19]:
# Tunggu hingga tombol Sign in muncul dan dapat diklik
try:
    print("Menunggu tombol Sign in muncul...")
    sign_in_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div[1]/div/div/div[3]/div[4]/a/div'))
    )
    
    print("Mengklik tombol Sign in...")
    sign_in_button.click()
    
    print("Berhasil mengklik tombol Sign in!")
except Exception as e:
    print(f"Terjadi kesalahan saat mengklik tombol Sign in: {e}")

Menunggu tombol Sign in muncul...
Mengklik tombol Sign in...
Berhasil mengklik tombol Sign in!


In [44]:
# Tunggu dan klik pada area input tweet dengan XPath yang benar
try:
    print("Menunggu area input tweet muncul...")
    tweet_input = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[1]/div[1]/div[1]/div/div/div/div/div[2]/div[2]/div/div/div/form/div[1]/div/div/div/div/div[2]/div/input'))
    )
    
    print("Mengklik area input tweet...")
    tweet_input.click()
    
    # Isi dengan teks "bakso"
    print("Mengisi teks 'bakso'...")
    tweet_input.send_keys("bakso")
    
    print("Berhasil mengisi teks!")
    
    # Tekan tombol Enter
    print("Menekan tombol Enter...")
    tweet_input.send_keys(webdriver.Keys.RETURN)
    
    print("Berhasil menekan Enter!")
    
    # Beri waktu untuk memastikan tweet terkirim
    time.sleep(3)
    
    # Klik pada elemen navigasi "Terbaru"
    print("Mengklik tab Terbaru...")
    nav_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div[1]/div[1]/div[2]/nav/div/div[2]/div/div[2]/a'))
    )
    
    nav_element.click()
    print("Berhasil mengklik tab Terbaru!")
    
    # Beri waktu untuk memuat tweet terbaru
    time.sleep(5)
    
    # Scan tweet dari posisi 1 hingga 7
    print("Memindai tweet terbaru...")
    tweets_data = []
    
    for i in range(1, 8):  # Dari 1 sampai 7
        try:
            # XPath dasar untuk artikel tweet
            tweet_article_xpath = f'//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[3]/section/div/div/div[{i}]/div[1]/div/article'
            
            # Tunggu artikel tweet muncul
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, tweet_article_xpath))
            )
            
            # Dapatkan elemen artikel tweet
            tweet_article = driver.find_element(By.XPATH, tweet_article_xpath)
            
            # Cari teks tweet, nama akun, dan waktu dalam artikel tweet
            try:
                # Cari teks tweet (menggunakan pendekatan yang lebih umum)
                tweet_text_elements = tweet_article.find_elements(By.CSS_SELECTOR, "div[data-testid='tweetText'] span")
                tweet_text = " ".join([elem.text for elem in tweet_text_elements if elem.text])
                
                # Jika tidak ada teks yang ditemukan, coba pendekatan lain
                if not tweet_text:
                    tweet_text = tweet_article.find_element(By.CSS_SELECTOR, "div[data-testid='tweetText']").text
            except:
                tweet_text = "Tidak dapat mengambil teks tweet"
            
            try:
                # Cari nama akun
                username_element = tweet_article.find_element(By.CSS_SELECTOR, "div[data-testid='User-Name'] span")
                username = username_element.text
            except:
                username = "Tidak dapat mengambil nama akun"
            
            try:
                # Cari waktu tweet
                time_element = tweet_article.find_element(By.CSS_SELECTOR, "time")
                tweet_time = time_element.get_attribute("datetime")
            except:
                tweet_time = "Tidak dapat mengambil waktu"
                
            try:
                # Cari tweet_id (menggunakan pendekatan yang lebih umum karena ID bisa berubah)
                # Coba dengan XPath yang diberikan terlebih dahulu
                tweet_id_elements = tweet_article.find_elements(By.XPATH, ".//div[2]/div/div[1]/a/div/span")
                if tweet_id_elements:
                    tweet_id = tweet_id_elements[0].text
                else:
                    # Alternatif: coba dengan CSS selector untuk username
                    tweet_id_elements = tweet_article.find_elements(By.CSS_SELECTOR, "div[data-testid='User-Name'] span.css-1qaijid")
                    if tweet_id_elements:
                        tweet_id = tweet_id_elements[0].text
                    else:
                        tweet_id = "Tidak dapat mengambil tweet_id"
            except:
                tweet_id = "Tidak dapat mengambil tweet_id"
            
            # Tambahkan data ke list
            tweets_data.append({
                'username': username,
                'tweet_id': tweet_id,
                'tweet_text': tweet_text,
                'tweet_time': tweet_time,
                'position': i
            })
            
            print(f"Tweet {i} berhasil dipindai.")
            
        except Exception as e:
            print(f"Gagal memindai tweet {i}: {e}")
    
    # Simpan data ke file CSV
    csv_filename = f"twitter_tweets_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['position', 'username', 'tweet_id', 'tweet_text', 'tweet_time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for tweet in tweets_data:
            writer.writerow(tweet)
    
    print(f"Data berhasil disimpan ke {csv_filename}")
    
    # Beri waktu untuk melihat hasil
    time.sleep(3)
    
    print("Proses selesai. Browser tetap terbuka.")
    
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

Menunggu area input tweet muncul...
Mengklik area input tweet...
Mengisi teks 'bakso'...
Berhasil mengisi teks!
Menekan tombol Enter...
Berhasil menekan Enter!
Mengklik tab Terbaru...
Berhasil mengklik tab Terbaru!
Memindai tweet terbaru...
Tweet 1 berhasil dipindai.
Tweet 2 berhasil dipindai.
Tweet 3 berhasil dipindai.
Tweet 4 berhasil dipindai.
Tweet 5 berhasil dipindai.
Tweet 6 berhasil dipindai.
Tweet 7 berhasil dipindai.
Data berhasil disimpan ke twitter_tweets_20250509_234126.csv
Proses selesai. Browser tetap terbuka.


In [ ]:
# Input dari pengguna
search_keyword = input("Masukkan kata kunci yang ingin dicari: ")
tweet_count = int(input("Masukkan jumlah tweet yang ingin diambil: "))


# Tunggu dan klik pada area input tweet dengan XPath yang benar
try:
    print("Menunggu area input tweet muncul...")
    tweet_input = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[1]/div[1]/div[1]/div/div/div/div/div[2]/div[2]/div/div/div/form/div[1]/div/div/div/div/div[2]/div/input'))
    )
    
    print("Mengklik area input tweet...")
    tweet_input.click()
    
    # Isi dengan kata kunci yang diinput pengguna
    print(f"Mengisi teks '{search_keyword}'...")
    tweet_input.send_keys(search_keyword)
    
    print("Berhasil mengisi teks!")
    
    # Tekan tombol Enter
    print("Menekan tombol Enter...")
    tweet_input.send_keys(webdriver.Keys.RETURN)
    
    print("Berhasil menekan Enter!")
    
    # Beri waktu untuk memastikan tweet terkirim
    time.sleep(3)
    
    # Klik pada elemen navigasi "Terbaru"
    print("Mengklik tab Terbaru...")
    nav_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div[1]/div[1]/div[2]/nav/div/div[2]/div/div[2]/a'))
    )
    
    nav_element.click()
    print("Berhasil mengklik tab Terbaru!")
    
    # Beri waktu untuk memuat tweet terbaru
    time.sleep(5)
    
    # Scan tweet sesuai jumlah yang diinput pengguna
    print(f"Memindai {tweet_count} tweet terbaru...")
    tweets_data = []
    
    for i in range(1, tweet_count + 1):  # Dari 1 sampai jumlah yang diinput
        try:
            # XPath dasar untuk artikel tweet
            tweet_article_xpath = f'//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[3]/section/div/div/div[{i}]/div[1]/div/article'
            
            # Tunggu artikel tweet muncul
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, tweet_article_xpath))
            )
            
            # Dapatkan elemen artikel tweet
            tweet_article = driver.find_element(By.XPATH, tweet_article_xpath)
            
            # Cari teks tweet, nama akun, dan waktu dalam artikel tweet
            try:
                # Cari teks tweet (menggunakan pendekatan yang lebih umum)
                tweet_text_elements = tweet_article.find_elements(By.CSS_SELECTOR, "div[data-testid='tweetText'] span")
                tweet_text = " ".join([elem.text for elem in tweet_text_elements if elem.text])
                
                # Jika tidak ada teks yang ditemukan, coba pendekatan lain
                if not tweet_text:
                    tweet_text = tweet_article.find_element(By.CSS_SELECTOR, "div[data-testid='tweetText']").text
            except:
                tweet_text = "Tidak dapat mengambil teks tweet"
            
            try:
                # Cari nama akun
                username_element = tweet_article.find_element(By.CSS_SELECTOR, "div[data-testid='User-Name'] span")
                username = username_element.text
            except:
                username = "Tidak dapat mengambil nama akun"
            
            try:
                # Cari waktu tweet
                time_element = tweet_article.find_element(By.CSS_SELECTOR, "time")
                tweet_time = time_element.get_attribute("datetime")
            except:
                tweet_time = "Tidak dapat mengambil waktu"
                
            try:
                # Cari tweet_id (menggunakan pendekatan yang lebih umum karena ID bisa berubah)
                # Coba dengan XPath yang diberikan terlebih dahulu
                tweet_id_elements = tweet_article.find_elements(By.XPATH, ".//div[2]/div/div[1]/a/div/span")
                if tweet_id_elements:
                    tweet_id = tweet_id_elements[0].text
                else:
                    # Alternatif: coba dengan CSS selector untuk username
                    tweet_id_elements = tweet_article.find_elements(By.CSS_SELECTOR, "div[data-testid='User-Name'] span.css-1qaijid")
                    if tweet_id_elements:
                        tweet_id = tweet_id_elements[0].text
                    else:
                        tweet_id = "Tidak dapat mengambil tweet_id"
            except:
                tweet_id = "Tidak dapat mengambil tweet_id"
            
            # Tambahkan data ke list
            tweets_data.append({
                'username': username,
                'tweet_id': tweet_id,
                'tweet_text': tweet_text,
                'tweet_time': tweet_time,
                'position': i
            })
            
            print(f"Tweet {i} berhasil dipindai.")
            
        except Exception as e:
            print(f"Gagal memindai tweet {i}: {e}")
    
    # Simpan data ke file CSV
    csv_filename = f"twitter_tweets_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['position', 'username', 'tweet_id', 'tweet_text', 'tweet_time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for tweet in tweets_data:
            writer.writerow(tweet)
    
    print(f"Data berhasil disimpan ke {csv_filename}")
    
    # Beri waktu untuk melihat hasil
    time.sleep(3)
    
    print("Proses selesai. Browser tetap terbuka.")
    
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

Menunggu tombol Sign in muncul...


KeyboardInterrupt: 